In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
 
sf_fire_file = "Files/sfcalls/sf-fire-calls.csv_0.csv"

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 3, Finished, Available, Finished)

In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 5, Finished, Available, Finished)

In [12]:
fireServiceCallsDF = spark.read.format("csv").option("header","true").schema(fire_schema).load("abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/sfcalls/sf-fire-calls.csv")

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 14, Finished, Available, Finished)

In [13]:
display(fireServiceCallsDF)

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6ca7eee1-2c0c-4c6e-abe6-73437ee04b6c)

In [14]:
fireServiceCallsDF.columns

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 16, Finished, Available, Finished)

['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallDate',
 'WatchDate',
 'CallFinalDisposition',
 'AvailableDtTm',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'Delay']

In [15]:
fireServiceCallsDF.count()

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 17, Finished, Available, Finished)

50000

In [16]:
display(fireServiceCallsDF.limit(5))

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f2b601f7-208c-4dee-a291-70cb32b2d4cb)

In [17]:
fireServiceCallsDF.select('CallType').distinct().show(35, False)

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 19, Finished, Available, Finished)

+-------------------------------+
|CallType                       |
+-------------------------------+
|Elevator / Escalator Rescue    |
|Alarms                         |
|Odor (Strange / Unknown)       |
|Citizen Assist / Service Call  |
|HazMat                         |
|Oil Spill                      |
|Vehicle Fire                   |
|Other                          |
|Outside Fire                   |
|Gas Leak (Natural and LP Gases)|
|Water Rescue                   |
|Electrical Hazard              |
|Structure Fire                 |
|Industrial Accidents           |
|Medical Incident               |
|Fuel Spill                     |
|Smoke Investigation (Outside)  |
|Aircraft Emergency             |
|Assist Police                  |
+-------------------------------+



In [18]:
display(fireServiceCallsDF.select('CallType').groupBy('CallType').count().orderBy("count", ascending=False))

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8338d583-059c-48f1-b6fd-a8476701c459)

In [20]:
fireServiceCallsDF.createOrReplaceTempView("firecalls")

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 22, Finished, Available, Finished)

In [22]:
%%sql

-- Modify this SQL here
select *
from firecalls
limit 5

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 24, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 28 fields>

In [4]:
%%spark
val path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/structured-streaming-blog/sdevices/"
mssparkutils.fs.mkdirs(path)
 
val numFiles = 100
val numDataPerFile = 100
 
import scala.util.Random
 
val deviceTypes = Seq("SensorTypeA", "SensorTypeB", "SensorTypeC", "SensorTypeD")
val startTime = System.currentTimeMillis
mssparkutils.fs.rm(path, true)
 
(1 to numFiles).par.foreach { fileId =>
  val file = s"$path/file-$fileId.json"
  val data = (1 to numDataPerFile).map { x => 
    val timestamp = new java.sql.Timestamp(startTime + (fileId * 60000) + (Random.nextInt() % 10000))
    val deviceId = Random.nextInt(100)
    val deviceType = deviceTypes(Random.nextInt(deviceTypes.size))
    val signalStrength = math.abs(Random.nextDouble % 100)
    s"""{"timestamp":"$timestamp","deviceId":$deviceId,"deviceType":"$deviceType","signalStrength":$signalStrength}"""
  }.mkString("\n")
  mssparkutils.fs.put(file, data)
}
mssparkutils.fs.head(mssparkutils.fs.ls(s"$path/file-1.json").head.path)


StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 7, Finished, Available, Finished)

path: String = abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/structured-streaming-blog/sdevices/
res8: Boolean = true
numFiles: Int = 100
numDataPerFile: Int = 100
import scala.util.Random
deviceTypes: Seq[String] = List(SensorTypeA, SensorTypeB, SensorTypeC, SensorTypeD)
startTime: Long = 1734372606922
res9: Boolean = true
res11: String =
{"timestamp":"2024-12-16 18:11:04.061","deviceId":71,"deviceType":"SensorTypeA","signalStrength":0.0237729895537655}
{"timestamp":"2024-12-16 18:11:03.814","deviceId":44,"deviceType":"SensorTypeD","signalStrength":0.1581196343492478}
{"timestamp":"2024-12-16 18:11:04.616","deviceId":65,"deviceType":"SensorTypeC","signalStrength":0.28166125470375536}
{"timestamp":"2024-12-16 18:11:10.656","deviceId":11,"deviceType":"SensorTypeC","signalStrength":0.5664958392874174}
{"timestamp":"2024-12-16 18:11:01.291","deviceId":97,"deviceType":"SensorTypeC","signalStrength":0.8351589853606657}
{"timestamp":"2024-12-16 18:10

In [5]:
output_path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/"
checkpoint_path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream-checkpoint"
#
#create checkpoint  path
#
#mssparkutils.fs.rm(checkpoint_path,True) #overwrite checkpoint
mssparkutils.fs.mkdirs(checkpoint_path)
#
#
bad_records_path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/badRecordsPath/streaming-sensor/"
#mssparkutils.fs.rm(bad_records_path, True) #empty dir
mssparkutils.fs.mkdirs(bad_records_path)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 8, Finished, Available, Finished)

True

In [6]:
sensor_path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/structured-streaming-blog/sdevices/"
sensor_file_name= sensor_path + "file-1.json"
mssparkutils.fs.head(sensor_file_name, 233)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 9, Finished, Available, Finished)

'{"timestamp":"2024-12-16 18:11:04.061","deviceId":71,"deviceType":"SensorTypeA","signalStrength":0.0237729895537655}\n{"timestamp":"2024-12-16 18:11:03.814","deviceId":44,"deviceType":"SensorTypeD","signalStrength":0.1581196343492478}'

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
 
#original input schema
jsonSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
)
# modified schema with added columns since we are 
# doing some ETL (transforming and adding extra columns)
# this transformed data will be stored into parquet files
# from which an SQL table can be created for consumption or
# report generation
parquetSchema = (
  StructType()
  .add("timestamp", TimestampType()) #event time at the source
  .add("deviceId", LongType())
  .add("deviceType", StringType())
  .add("signalStrength", DoubleType())
  .add("INPUT_FILE_NAME", StringType()) #file name from which this data item was read
  .add("PROCESSED_TIME", TimestampType())) #time at the executor while processing


StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 10, Finished, Available, Finished)

In [ ]:
abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files

In [8]:
inputDF = ( spark 
          .readStream 
          .schema(jsonSchema) 
          .option("maxFilesPerTrigger", 1)  #slow it down for tutorial
          .option("badRecordsPath", bad_records_path) #any bad records will go here
          .json(sensor_path) #the source
          .withColumn("INPUT_FILE_NAME", input_file_name()) #maintain file path
          .withColumn("PROCESSED_TIME", current_timestamp()) #add a processing timestamp at the time of processing
          .withWatermark("PROCESSED_TIME", "1 minute") #optional: window for out of order data
         )

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 11, Finished, Available, Finished)

In [23]:
delta_stream_table_path = 'Tables/iotdevicedata'
query = (inputDF
         .writeStream
         .format("delta") #our sink to save it for posterity or batch queries if needed
         .option("checkpointLocation", checkpoint_path) # add checkpointing for resiliency
         .start(delta_stream_table_path) 
        )

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 26, Finished, Available, Finished)

In [25]:
query = (inputDF
         .writeStream
         .format("parquet") #our sink to save it for posterity or batch queries if needed
         .option("path", output_path)
         .option("checkpointLocation", checkpoint_path) # add checkpointing for resiliency
         .outputMode("append")
         .queryName("devicesquery") #optionally a query name over write to issue queries against
         .trigger(processingTime='5 seconds')
         .start() 
        )

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 28, Finished, Available, Finished)

In [61]:
%fs ls "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/"

StatementMeta(, f3c0e2f5-400b-4593-9fa2-80d7cc605a8a, 64, Finished, Available, Finished)

FileInfo(_spark_metadata, abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/_spark_metadata, 0)
FileInfo(part-00000-0402d0db-347e-4d53-a842-e79a00a06980-c000.snappy.parquet, abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/part-00000-0402d0db-347e-4d53-a842-e79a00a06980-c000.snappy.parquet, 6162)
FileInfo(part-00000-04c7712f-b58d-4e4e-b51a-c88b83d19615-c000.snappy.parquet, abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/part-00000-04c7712f-b58d-4e4e-b51a-c88b83d19615-c000.snappy.parquet, 6134)
FileInfo(part-00000-06b902ff-c3e0-4dc0-854d-e9644118d07f-c000.snappy.parquet, abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/tmp/Streaming/continuous_streaming/out/iot-stream/part-00000-06b902ff-c3e0-4dc0-854d

In [26]:
%%sql
CREATE TABLE IF NOT EXISTS firecalls_delta
  (CallNumber INT, IncidentNumber Int, CallType STRING,CallDate STRING,Address STRING,City STRING,Delay DOUBLE)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 29, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [27]:
%%sql
INSERT INTO firecalls_delta
VALUES 
  (101, 5001, "Medical","01/11/2002","10TH ST/MARKET ST","SF",5.233333),
  (102, 5002, "Fire","01/12/2002","2000 Block of CALIFORNIA ST","SF",3.433),
  (104, 5005, "Structure Fire","01/16/2002","2000 Block of CALIFORNIA ST","SF",2.54)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 30, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
%%sql
SELECT * FROM firecalls_delta

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 31, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 7 fields>

In [29]:
%%sql
DESCRIBE EXTENDED firecalls_delta

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 32, Finished, Available, Finished)

<Spark SQL result set with 15 rows and 3 fields>

In [30]:
%%sql
DESCRIBE DETAIL firecalls_delta

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 33, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 15 fields>

In [31]:
display(mssparkutils.fs.ls(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta"))

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 34, Finished, Available, Finished)

[FileInfo(path=abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/part-00000-1acbb706-2ba2-4361-bf0a-a56446809ba2-c000.snappy.parquet, name=part-00000-1acbb706-2ba2-4361-bf0a-a56446809ba2-c000.snappy.parquet, size=3599)]

In [33]:
display(mssparkutils.fs.ls(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log"))

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 36, Finished, Available, Finished)

[FileInfo(path=abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log/00000000000000000000.json, name=00000000000000000000.json, size=1243),
 FileInfo(path=abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log/00000000000000000001.json, name=00000000000000000001.json, size=1129),
 FileInfo(path=abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log/_commits, name=_commits, size=0)]

In [34]:
display(spark.sql(f"SELECT * FROM json.`abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/firecalls_delta/_delta_log/00000000000000000000.json`"))

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 421f5e62-88e2-43be-beaa-98d1ae5ab2ad)

In [35]:
%%sql
DESCRIBE HISTORY firecalls_delta

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 38, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

In [36]:
%%sql
UPDATE firecalls_delta 
SET Delay = Delay + 1
WHERE CallType LIKE "F%"

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 39, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [38]:
%%sql
DESCRIBE HISTORY firecalls_delta

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 41, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 15 fields>

In [39]:
%%sql
SELECT * 
FROM firecalls_delta VERSION AS OF 1

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 42, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 7 fields>

In [40]:
%%sql
CREATE TABLE fc_external_table
USING CSV OPTIONS (
  path = 'abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/sfcalls/sf-fire-calls.csv',
  header = "true",
  mode = "FAILFAST"
);
 
SELECT * FROM fc_external_table;
 

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 44, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 28 fields>

In [41]:
%%sql
CREATE VIEW view_firecall_medicalinc AS
  SELECT * 
  FROM fc_external_table 
  WHERE CallType = 'Medical Incident' AND City = 'SF';
 
SELECT * FROM view_firecall_medicalinc;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 46, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 28 fields>

In [42]:
%%sql
SHOW TABLES;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 47, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 3 fields>

In [43]:
%%sql
CREATE TEMPORARY VIEW temp_view_fc_delays_gt_3
AS SELECT * FROM fc_external_table WHERE delay > 3 ORDER BY delay ASC;
 
SELECT * FROM temp_view_fc_delays_gt_3;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 49, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 28 fields>

In [44]:
%%sql
SHOW TABLES;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 50, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>

In [45]:
%%sql
CREATE GLOBAL TEMPORARY VIEW global_temp_view_priority_gte_3 
AS SELECT * FROM fc_external_table WHERE Priority >= 3;
 
SELECT * FROM global_temp.global_temp_view_priority_gte_3;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 52, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1000 rows and 28 fields>

In [46]:
%%sql
SHOW TABLES;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 53, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>

In [47]:
%%sql
SHOW TABLES IN global_temp;

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 54, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 3 fields>

In [54]:

from pyspark.sql.types import *
from pyspark.sql.functions import *
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 63, Finished, Available, Finished)

In [55]:
file_path = "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/sfcalls/sf-fire-calls.csv"
 
fireServiceCallsDF = spark.read.csv(file_path, header=True, schema=fire_schema)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 64, Finished, Available, Finished)

Ingest_Bronze

In [59]:
(
  fireServiceCallsDF
  .write
  .mode("overwrite")
  .format("delta")
  .save(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/bronze")
)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 68, Finished, Available, Finished)

In [61]:
spark.sql(
f"""
  CREATE TABLE fire_calls_bronze1
  USING DELTA
  LOCATION "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/bronze"
"""
)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 70, Finished, Available, Finished)

DataFrame[]

In [62]:
df = spark.sql("SELECT * FROM datastore.fire_calls_bronze1 LIMIT 1000")
display(df)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 71, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eaed6346-9e7e-4fda-a1aa-5d6ee3e3cebc)

In [65]:
fire_calls_bronze_df = spark.read.format("delta").load(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/bronze")
display(fire_calls_bronze_df)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f905c629-5287-4b53-8957-13f8e0540071)

In [66]:
fire_calls_silver_df = fire_calls_bronze_df.select(
  col("IncidentNumber").cast("integer").alias("IncidentNumber"),
    "CallType",
   "CallDate",
  "WatchDate",
    "NumAlarms",
    "Delay"
    
    
    
)
 

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 75, Finished, Available, Finished)

In [67]:
display(fire_calls_silver_df)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 76, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0d49748e-7dc7-49f5-8cfa-e8a6348b6951)

In [76]:
(
    fire_calls_silver_df.write.mode("overwrite")
    .format("delta")
    .partitionBy("CallType")
    .save(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/silver")
)
 
spark.sql(
  f"""
    CREATE TABLE IF NOT EXISTS fire_calls_silver
    USING DELTA
    LOCATION "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/silver"
  """
)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 85, Finished, Available, Finished)

DataFrame[]

In [79]:
fire_calls_silver_df = spark.read.format("delta").load(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/silver")
display(fire_calls_silver_df)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 88, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 272c91d4-0271-428d-b2dc-82ec540c2cb0)

In [80]:
from pyspark.sql.functions import col, avg, max, stddev
 
fire_calls_gold_df = (
  fire_calls_silver_df
  .groupby("CallType")
  .agg(avg(col("Delay")).alias("avg_ResponceDelay"),
       max(col("Delay")).alias("max_ResponceDelay"),
       stddev(col("Delay")).alias("stddev_ResponceDelay"))
)
 
display(fire_calls_gold_df)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 89, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 414971d7-d2f6-4b17-b7e1-ca65656f8a9b)

In [81]:
(
    fire_calls_gold_df.write.mode("overwrite")
    .format("delta")
    .save(f"abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Files/gold")
)
 
spark.sql(
  f"""
    CREATE TABLE IF NOT EXISTS fire_calls_gold
    USING DELTA
    LOCATION "abfss://trainerworkspace@onelake.dfs.fabric.microsoft.com/datastore.Lakehouse/Tables/gold"
  """
)

StatementMeta(, 29208996-e67a-4919-8ac4-0c5cede7eda4, 90, Finished, Available, Finished)

DataFrame[]